In [1]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *
import re

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client.adsE_database
collection = db.adsE_collection

In [5]:
def makejson(file):
    atoms = query(file, 'spacom')

    dic = {}

    res = re.match('(.*u\d).*', file)
    barefile = res.group(1) + '.traj'
    bareatoms = query(barefile, 'spacom')
#     print(bareatoms)

    COatoms = query('CO.traj', 'scpacom')
    COene = COatoms.get_potential_energy()

    res = re.match('(\w.*)_(\d{3}).*', file)
    ele = res.group(1)
    mir = res.group(2)
    
    num = 0
    ads = None
    res = re.match('.*n(\d+)_', file)
    if res:
        num = int(res.group(1))
        
    res = re.match('.*u(\d+).*', file)
    unit = int(res.group(1))

    res = re.match('.*no.*_(\w.)_n', file)
    if res:
        ads = res.group(1)

    ### calc area ###
    x = atoms.cell[0][0]
    y = atoms.cell[1][1]
    area = x*y

    ### calc adsorvates number ###
    tot = 0
    for atom in atoms:
        if atom.tag == 1:
            tot += 1

    ### get energies ###
    ene = atoms.get_potential_energy()
    bareene = bareatoms.get_potential_energy()
    totaladsene = ene - (COene*num + bareene)
    
    ### get adsorbate position info ###
    igroups, iposlis, rgroups, rposlis = getadsposinfo(file, 0)
    
    ### get COint info ###
    try:
        file_ = file[:-5] + '__.traj'
        atoms_ = query(file_, 'spacom')
        ene_ = atoms_.get_potential_energy()
    except:
        ene_ = 0
        
    ### get COlength info ###
    COlengthlis = []
    for i in range(len(atoms)):
        if atoms[i].symbol == 'C':
            Cpos = atoms[i].position
            Opos = atoms[i+1].position
            COlengthlis.append(np.linalg.norm(Cpos-Opos))
    
    ### get convergence ###
    converged = np.max(np.abs(atoms.get_forces())) < 0.03
        
    dic['name'] = file
    dic['element'] = ele
    dic['mirror'] = mir
    dic['adsorbate'] = ads
    dic['unitlength'] = unit
    dic['surfatomnum'] = tot
    dic['area'] = area
    dic['Energy'] = ene
    dic['totaladsEnergy'] = totaladsene
    dic['aveadsEne/suratom'] = totaladsene/tot
    dic['numberofads'] = num
    dic['coverage'] = num/tot
    dic['density'] = num/area
    dic['igroups'] = igroups
    dic['iposlis'] = [list(item) for item in iposlis]
    dic['rgroups'] = rgroups
    dic['rposlis'] = [list(item) for item in rposlis]
    dic['ECO_Eint'] = ene_
    dic['COlengthlis'] = COlengthlis
    dic['converged'] = 'yes' if converged else 'no'
    
    dic['aveadsEne/ads'] = totaladsene/num if num else None

    maxdiff = getmaxdiff(file)
    dic['moved+'] = 'yes' if maxdiff > 1.5 else 'no'
    dic['moved'] = 'no' if igroups == rgroups else 'yes'
    
    for key in dic.keys():
        print('{}: {}'.format(key, dic[key]))
    
    return dic

In [6]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')
for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if '__' not in files[i]:
                print(i, files[i])

3 Ag_100_u1.traj
5 Ag_100_u1_no01_CO_n1_d3.traj
9 Ag_100_u1_no02_CO_n1_d3.traj
13 Ag_100_u1_no03_CO_n1_d3.traj
17 Ag_100_u2.traj
19 Ag_100_u2_no01_CO_n1_d9.traj
23 Ag_100_u2_no02_CO_n2_d3.traj
27 Ag_100_u2_no03_CO_n3_d3.traj
31 Ag_100_u2_no04_CO_n4_d3.traj
35 Ag_100_u2_no05_CO_n3_d3.traj
39 Ag_100_u2_no06_CO_n4_d3.traj
43 Ag_100_u2_no07_CO_n2_d6.traj
47 Ag_100_u2_no08_CO_n2_d4.traj
51 Ag_100_u2_no09_CO_n3_d3.traj
55 Ag_100_u2_no10_CO_n1_d9.traj
59 Ag_100_u2_no11_CO_n2_d3.traj
63 Ag_100_u2_no12_CO_n3_d3.traj
67 Ag_100_u2_no13_CO_n4_d3.traj
71 Ag_100_u2_no14_CO_n3_d3.traj
75 Ag_100_u2_no15_CO_n4_d3.traj
79 Ag_100_u2_no16_CO_n2_d6.traj
83 Ag_100_u2_no17_CO_n2_d4.traj
87 Ag_100_u2_no18_CO_n3_d3.traj
91 Ag_100_u2_no19_CO_n2_d3.traj
95 Ag_100_u2_no20_CO_n1_d9.traj
99 Ag_100_u2_no21_CO_n2_d3.traj
103 Ag_100_u2_no22_CO_n3_d3.traj
107 Ag_100_u2_no23_CO_n4_d3.traj
111 Ag_100_u2_no24_CO_n2_d6.traj
115 Ag_100_u3.traj
117 Ag_100_u3_no01_CO_n1_d15.traj
121 Ag_100_u3_no02_CO_n1_d15.traj
125 Ag_100_u3

In [7]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')

for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if 'CO.traj' not in files[i]:
                if '__' not in files[i]:
                    print(i, files[i])

                    post_data = makejson(files[i])
                    result = collection.insert_one(post_data)
                    print('One post: {0}\n'.format(result.inserted_id))

3 Ag_100_u1.traj
Symmetrically same structure found!
name: Ag_100_u1.traj
element: Ag
mirror: 100
adsorbate: None
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -8.12811404
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
coverage: 0.0
density: 0.0
igroups: []
iposlis: []
rgroups: []
rposlis: []
ECO_Eint: 0
COlengthlis: []
converged: yes
aveadsEne/ads: None
moved+: no
moved: no
One post: 5de53d6b237123b6eeb97811

5 Ag_100_u1_no01_CO_n1_d3.traj
Symmetrically same structure found!
name: Ag_100_u1_no01_CO_n1_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -22.44298044
totaladsEnergy: 0.1705699100000011
aveadsEne/suratom: 0.1705699100000011
numberofads: 1
coverage: 1.0
density: 0.11267657015369789
igroups: [0]
iposlis: [[0.0, 0.0, 18.169597920855004]]
rgroups: [0]
rposlis: [[5.521685380174929e-07, 5.944197952180715e-07, 18.562673073888003]]
ECO_Eint: -14.2275679
COlengthlis: [1.1576383705845288]
converged: ye

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no05_CO_n3_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -75.57290444
totaladsEnergy: 0.3548707300000018
aveadsEne/suratom: 0.08871768250000045
numberofads: 3
coverage: 0.75
density: 0.08450742761527343
igroups: [1, 0, 0]
iposlis: [[1.489543514736326, 2.979087029472652, 18.169597920855004], [2.979087029472652, 0.0, 18.169597920855004], [0.0, 0.0, 18.169597920855004]]
rgroups: [1, 0, 0]
rposlis: [[1.4896291924741782, 2.979090970553484, 18.109636220801

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no12_CO_n3_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -56.06735053
totaladsEnergy: 19.860424640000005
aveadsEne/suratom: 4.965106160000001
numberofads: 3
coverage: 0.75
density: 0.08450742761527343
igroups: [1, 1, 1]
iposlis: [[2.6645352591003757e-15, 1.4895435147363276, 18.169597920855004], [0.0, 4.468630544208981, 18.169597920855004], [2.979087029472652, 4.468630544208981, 18.169597920855004]]
rgroups: [1, 1, 1]
rposlis: [[-0.0011167432233223464

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no18_CO_n3_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -75.58394155
totaladsEnergy: 0.34383361999999806
aveadsEne/suratom: 0.08595840499999952
numberofads: 3
coverage: 0.75
density: 0.08450742761527343
igroups: [2, 2, 1]
iposlis: [[1.489543514736326, 1.4895435147363274, 18.169597920855004], [4.468630544208981, 1.4895435147363276, 18.169597920855004], [2.979087029472652, 4.468630544208981, 18.169597920855004]]
rgroups: [2, 2, 1]
rposlis: [[1.4709045

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Ag_100_u2_no24_CO_n2_d6.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -61.43607326
totaladsEnergy: 0.006265599999998983
aveadsEne/suratom: 0.0015663999999997458
numberofads: 2
coverage: 0.5
density: 0.056338285076848946
igroups: [2, 2]
iposlis: [[4.468630544208981, 1.4895435147363276, 18.169597920855004], [1.489543514736326, 4.468630544208981, 18.169597920855004]]
rgroups: [2, 2]
rposlis: [[4.465774551705367, 1.4912931028913754, 20.117168970147276], [1.4869353457116807, 4.470317558126378, 20.117818458159046]]
ECO_Eint: -28.99176022
COlengthlis: [1.1488470438202139, 1.148500200577781]
converged: no
aveadsEne/ads: 0.0031327999999994915
moved+: yes
moved: no
One post: 5de53d7b237123b6eeb9782d

115 Ag_100_u3.traj
Symmetrically same structure found!
Symmetrically same structure found!


...

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Zn_0001_u2_no017_CO_n3_d3.traj
element: Zn
mirror: 000
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 25.50758939991318
Energy: -53.30989623
totaladsEnergy: 1.5166831000000087
aveadsEne/suratom: 0.3791707750000022
numberofads: 3
coverage: 0.75
density: 0.11761205470910596

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Zn_0001_u2_no022_CO_n3_d3.traj
element: Zn
mirror: 000
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 25.50758939991318
Energy: -52.9049247
totaladsEnergy: 1.9216546300000061
aveadsEne/suratom: 0.48041365750000153
numberofads: 3
coverage: 0.75
density: 0.11761205470910596
igroups: [3, 3, 3]
iposlis: [[4.0703397041549945, 3.916686206478518, 19.275026588370004], [5.427119605539999, 1.5666744825914076, 19.275026588370004], [2.7135598027700003, 1.5666744825914067, 19.275026588370004]]
rgroups: [3, 3, 3]
rposlis: [[4.071363597497368, 3.9655523940329984, 20.494743183448676], [5.471088564819856, 1.540571209888754, 20.49266225158133

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure